Primera versión: sin memoria compartida

In [ ]:
%%writefile stenciltest.cu

#include <stdio.h>

#define RADIUS        3
#define BLOCK_SIZE    256
#define NUM_ELEMENTS  (4096*2)

// CUDA API error checking macro
#define cudaCheck(error) \
  if (error != cudaSuccess) { \
    printf("Fatal error: %s at %s:%d\n", \
      cudaGetErrorString(error), \
      __FILE__, __LINE__); \
    exit(1); \
  }

__global__ void stencil_1d(int *in, int *out) 
{
    int index = threadIdx.x + (blockIdx.x * blockDim.x) + RADIUS;

    // Apply the stencil
    int result = 0;
    for (int offset = -RADIUS ; offset <= RADIUS ; offset++)
        result += in[index + offset];

    // Store the result
    out[index-RADIUS] = result;
}

int main()
{
  unsigned int i;
  int h_in[NUM_ELEMENTS + 2 * RADIUS], h_out[NUM_ELEMENTS];
  int *d_in, *d_out;

  // Initialize host data
  for( i = 0; i < (NUM_ELEMENTS + 2*RADIUS); ++i )
    h_in[i] = 1; // With a value of 1 and RADIUS of 3, all output values should be 7

  // Allocate space on the device
  cudaCheck( cudaMalloc( &d_in, (NUM_ELEMENTS + 2*RADIUS) * sizeof(int)) );
  cudaCheck( cudaMalloc( &d_out, NUM_ELEMENTS * sizeof(int)) );

  // Copy input data to device
  cudaCheck( cudaMemcpy( d_in, h_in, (NUM_ELEMENTS + 2*RADIUS) * sizeof(int), cudaMemcpyHostToDevice) );

  stencil_1d<<< (NUM_ELEMENTS + BLOCK_SIZE - 1)/BLOCK_SIZE, BLOCK_SIZE >>> (d_in, d_out);

  cudaCheck( cudaMemcpy( h_out, d_out, NUM_ELEMENTS * sizeof(int), cudaMemcpyDeviceToHost) );

  // Verify every out value is the expected output
  for( i = 0; i < NUM_ELEMENTS; ++i )
    if (h_out[i] != (2*RADIUS + 1))
    {
      printf("Element h_out[%d] == %d != %d\n", i, h_out[i],(2*RADIUS + 1));
      break;
    }

  if (i == NUM_ELEMENTS)
    printf("SUCCESS!\n");

  // Free out memory
  cudaFree(d_in);
  cudaFree(d_out);

  return 0;
}

Overwriting stenciltest.cu


In [ ]:
!/usr/local/cuda/bin/nvcc -arch=sm_35 -rdc=true stenciltest.cu -o ./stenciltest -lcudadevrt
!nvprof ./stenciltest

nvcc warning : The 'compute_35', 'compute_37', 'compute_50', 'sm_35', 'sm_37' and 'sm_50' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).
==396== NVPROF is profiling process 396, command: ./stenciltest
==396== Warning: Auto boost enabled on device 0. Profiling results may be inconsistent.
SUCCESS!
==396== Profiling application: ./stenciltest
==396== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   44.57%  9.0560us         1  9.0560us  9.0560us  9.0560us  [CUDA memcpy HtoD]
                   33.39%  6.7840us         1  6.7840us  6.7840us  6.7840us  [CUDA memcpy DtoH]
                   22.05%  4.4800us         1  4.4800us  4.4800us  4.4800us  stencil_1d(int*, int*)
      API calls:   99.45%  199.90ms         2  99.952ms  8.5570us  199.90ms  cudaMalloc
                    0.28%  555.74us         1  555.74us  555.74us  555.74us  cuDeviceTotalMe

In [ ]:
!nvidia-smi

Sun Nov  7 21:34:19 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   74C    P0    74W / 149W |      0MiB / 11441MiB |      6%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Segunda versión: Memoria compartida pero sin usar syncthreads

In [ ]:
%%writefile stencilshared.cu

#include <stdio.h>

#define RADIUS        3
#define BLOCK_SIZE    256
#define NUM_ELEMENTS  (4096*2)

// CUDA API error checking macro
#define cudaCheck(error) \
  if (error != cudaSuccess) { \
    printf("Fatal error: %s at %s:%d\n", \
      cudaGetErrorString(error), \
      __FILE__, __LINE__); \
    exit(1); \
  }

__global__ void stencil_1d(int *in, int *out) 
{
    __shared__ int temp[BLOCK_SIZE + 2 * RADIUS];
    int gindex = threadIdx.x + (blockIdx.x * blockDim.x) + RADIUS;
    int lindex = threadIdx.x + RADIUS;

    // Read input elements into shared memory
    temp[lindex] = in[gindex];
    if (threadIdx.x < RADIUS) 
    {
        temp[lindex - RADIUS] = in[gindex - RADIUS];
        temp[lindex + BLOCK_SIZE] = in[gindex + BLOCK_SIZE];
    }

    // Make sure all threads get to this point before proceeding!
    //__syncthreads();

    // Apply the stencil
    int result = 0;
    for (int offset = -RADIUS ; offset <= RADIUS ; offset++)
        result += temp[lindex + offset];

    // Store the result
    out[gindex-RADIUS] = result;
}

int main()
{
  unsigned int i;
  int h_in[NUM_ELEMENTS + 2 * RADIUS], h_out[NUM_ELEMENTS];
  int *d_in, *d_out;

  // Initialize host data
  for( i = 0; i < (NUM_ELEMENTS + 2*RADIUS); ++i )
    h_in[i] = 1; // With a value of 1 and RADIUS of 3, all output values should be 7

  // Allocate space on the device
  cudaCheck( cudaMalloc( &d_in, (NUM_ELEMENTS + 2*RADIUS) * sizeof(int)) );
  cudaCheck( cudaMalloc( &d_out, NUM_ELEMENTS * sizeof(int)) );

  // Copy input data to device
  cudaCheck( cudaMemcpy( d_in, h_in, (NUM_ELEMENTS + 2*RADIUS) * sizeof(int), cudaMemcpyHostToDevice) );

  stencil_1d<<< (NUM_ELEMENTS + BLOCK_SIZE - 1)/BLOCK_SIZE, BLOCK_SIZE >>> (d_in, d_out);

  cudaCheck( cudaMemcpy( h_out, d_out, NUM_ELEMENTS * sizeof(int), cudaMemcpyDeviceToHost) );

  // Verify every out value is 7
  for( i = 0; i < NUM_ELEMENTS; ++i )
    if (h_out[i] != 7)
    {
      printf("Element h_out[%d] == %d != 7\n", i, h_out[i]);
      break;
    }

  //if (i == NUM_ELEMENTS)
  //  printf("SUCCESS!\n");

  // Free out memory
  cudaFree(d_in);
  cudaFree(d_out);

  return 0;
}

Writing stencilshared.cu


In [ ]:
!/usr/local/cuda/bin/nvcc -arch=sm_35 -rdc=true stencilshared.cu -o ./stencilshared -lcudadevrt
!nvprof ./stencilshared

nvcc warning : The 'compute_35', 'compute_37', 'compute_50', 'sm_35', 'sm_37' and 'sm_50' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).
==457== NVPROF is profiling process 457, command: ./stencilshared
==457== Warning: Auto boost enabled on device 0. Profiling results may be inconsistent.
==457== Profiling application: ./stencilshared
==457== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   46.92%  9.0240us         1  9.0240us  9.0240us  9.0240us  [CUDA memcpy HtoD]
                   35.27%  6.7840us         1  6.7840us  6.7840us  6.7840us  [CUDA memcpy DtoH]
                   17.80%  3.4240us         1  3.4240us  3.4240us  3.4240us  stencil_1d(int*, int*)
      API calls:   99.44%  198.32ms         2  99.159ms  5.0320us  198.31ms  cudaMalloc
                    0.31%  608.84us         1  608.84us  608.84us  608.84us  cuDeviceTotalMem
   

In [ ]:
!nvprof ./stencilshared

======== Error: application not found.


Tercera versión: con Syncthreads

In [ ]:
%%writefile stencilshared_sync.cu

#include <stdio.h>

#define RADIUS        3
#define BLOCK_SIZE    256
#define NUM_ELEMENTS  (4096*2)

// CUDA API error checking macro
#define cudaCheck(error) \
  if (error != cudaSuccess) { \
    printf("Fatal error: %s at %s:%d\n", \
      cudaGetErrorString(error), \
      __FILE__, __LINE__); \
    exit(1); \
  }

__global__ void stencil_1d(int *in, int *out) 
{
    __shared__ int temp[BLOCK_SIZE + 2 * RADIUS];
    int gindex = threadIdx.x + (blockIdx.x * blockDim.x) + RADIUS;
    int lindex = threadIdx.x + RADIUS;
 

    //Read input elements into shared memory
    temp[lindex] = in[gindex];
    if (threadIdx.x < RADIUS) 
    {
        temp[lindex - RADIUS] = in[gindex - RADIUS];
        temp[lindex + BLOCK_SIZE] = in[gindex + BLOCK_SIZE];
    }

    // Make sure all threads get to this point before proceeding!
    __syncthreads();

    // Apply the stencil
    int result = 0;
    for (int offset = -RADIUS ; offset <= RADIUS ; offset++)
        result += temp[lindex + offset];

    // Store the result
    out[gindex-RADIUS] = result;
}

int main()
{
  unsigned int i;
  int h_in[NUM_ELEMENTS + 2 * RADIUS], h_out[NUM_ELEMENTS];
  int *d_in, *d_out;

  // Initialize host data
  for( i = 0; i < (NUM_ELEMENTS + 2*RADIUS); ++i )
    h_in[i] = 1; // With a value of 1 and RADIUS of 3, all output values should be 7

  // Allocate space on the device
  cudaCheck( cudaMalloc( &d_in, (NUM_ELEMENTS + 2*RADIUS) * sizeof(int)) );
  cudaCheck( cudaMalloc( &d_out, NUM_ELEMENTS * sizeof(int)) );

  // Copy input data to device
  cudaCheck( cudaMemcpy( d_in, h_in, (NUM_ELEMENTS + 2*RADIUS) * sizeof(int), cudaMemcpyHostToDevice) );

  stencil_1d<<< (NUM_ELEMENTS + BLOCK_SIZE - 1)/BLOCK_SIZE, BLOCK_SIZE >>> (d_in, d_out);

  cudaCheck( cudaMemcpy( h_out, d_out, NUM_ELEMENTS * sizeof(int), cudaMemcpyDeviceToHost) );

  // Verify every out value is (2*RADIUS + 1)
  for( i = 0; i < NUM_ELEMENTS; ++i )
    if (h_out[i] != (2*RADIUS + 1))
    {
      printf("Element h_out[%d] == %d != %d\n", i, h_out[i],(2*RADIUS + 1));
      break;
    }

  //if (i == NUM_ELEMENTS)
   // printf("SUCCESS!\n");

  // Free out memory
  cudaFree(d_in);
  cudaFree(d_out);

  return 0;
}

Writing stencilshared_sync.cu


In [ ]:
!/usr/local/cuda/bin/nvcc -arch=sm_35 -rdc=true stencilshared_sync.cu -o ./stencilshared_sync -lcudadevrt
!nvprof ./stencilshared_sync

nvcc warning : The 'compute_35', 'compute_37', 'compute_50', 'sm_35', 'sm_37' and 'sm_50' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).
==257== NVPROF is profiling process 257, command: ./stencilshared_sync
==257== Warning: Auto boost enabled on device 0. Profiling results may be inconsistent.
==257== Warning: Profiling results might be incorrect with current version of nvcc compiler used to compile cuda app. Compile with nvcc compiler 9.0 or later version to get correct profiling results. Ignore this warning if code is already compiled with the recommended nvcc version 
==257== Profiling application: ./stencilshared_sync
==257== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   45.75%  9.1200us         1  9.1200us  9.1200us  9.1200us  [CUDA memcpy HtoD]
                   34.19%  6.8160us         1  6.8160us  6.8160us  6.8160us  [CUDA memcp

In [ ]:
%%writefile o_stencil_np.cu
#include <iostream>
#include <algorithm>
using namespace std;

#define N (4096*2)
#define RADIUS 3
#define BLOCK_SIZE 256
__global__ void stencil_1d(int *in, int *out) {
    // Índice global de la posición central de los datos que va a usar el thread
    int index = threadIdx.x + (blockIdx.x * blockDim.x) + RADIUS;

    // Realizamos la operación del stencil
    int result = 0;
    for (int offset = -RADIUS; offset <= RADIUS; offset++)
        result += in[index + offset];

    // Guardamos el resultado
    out[index-RADIUS] = result;
}
void fill_ints(int *x, int n)
{
	fill_n(x, n, 1);
}
int main(void)
{
	int *in, *out;	// host copies of a, b, c
	int *d_in, *d_out;	// device copies of a, b, c
	int size = (N + 2 *RADIUS) *sizeof(int);
	// Alloc space for host copies and setup values
	in = (int*) malloc(size);
	fill_ints(in, N + 2 *RADIUS);
	out = (int*) malloc(size);
	fill_ints(out, N + 2 *RADIUS);
	// Alloc space for device copies
	cudaMalloc((void **) &d_in, size);
	cudaMalloc((void **) &d_out, size);
	// Copy to device
	cudaMemcpy(d_in, in, size, cudaMemcpyHostToDevice);
	cudaMemcpy(d_out, out, size, cudaMemcpyHostToDevice);
	// Launch stencil_1d() kernel on GPU
	stencil_1d <<<(N + BLOCK_SIZE - 1 ) / BLOCK_SIZE, BLOCK_SIZE>>> (d_in + RADIUS, d_out + RADIUS);
	// Copy result back to host
	cudaMemcpy(out, d_out, size, cudaMemcpyDeviceToHost);
	// Cleanup
	free(in);
	free(out);
	cudaFree(d_in);
	cudaFree(d_out);
	return 0;
}

Writing o_stencil_np.cu


In [ ]:
!/usr/local/cuda/bin/nvcc -arch=sm_35 -rdc=true o_stencil_np.cu -o ./o_stencil_np -lcudadevrt
!nvprof ./o_stencil_np

nvcc warning : The 'compute_35', 'compute_37', 'compute_50', 'sm_35', 'sm_37' and 'sm_50' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).
==567== NVPROF is profiling process 567, command: ./o_stencil_np
==567== Warning: Auto boost enabled on device 0. Profiling results may be inconsistent.
==567== Profiling application: ./o_stencil_np
==567== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   61.29%  17.984us         2  8.9920us  8.8640us  9.1200us  [CUDA memcpy HtoD]
                   23.23%  6.8160us         1  6.8160us  6.8160us  6.8160us  [CUDA memcpy DtoH]
                   15.49%  4.5440us         1  4.5440us  4.5440us  4.5440us  stencil_1d(int*, int*)
      API calls:   99.47%  199.51ms         2  99.757ms  16.964us  199.50ms  cudaMalloc
                    0.26%  513.14us         1  513.14us  513.14us  513.14us  cuDeviceTotalMem
     

In [ ]:
%%writefile o_stencil.cu
#include <iostream>
#include <algorithm>
using namespace std;

#define N (4096*2)
#define RADIUS 3
#define BLOCK_SIZE 256
__global__ void stencil_1d(int *in, int *out)
{
	__shared__ int temp[BLOCK_SIZE + 2 *RADIUS];
	int gindex = threadIdx.x + blockIdx.x *blockDim.x;
	int lindex = threadIdx.x + RADIUS;
	// Read input elements into shared memory
	temp[lindex] = in[gindex];
	if (threadIdx.x < RADIUS)
	{
		temp[lindex - RADIUS] = in[gindex - RADIUS];
		temp[lindex + BLOCK_SIZE] = in[gindex + BLOCK_SIZE];
	}
	// Synchronize (ensure all the data is available)
	//__syncthreads();
	// Apply the stencil
	int result = 0;
	for (int offset = -RADIUS; offset<= RADIUS; offset++)
		result += temp[lindex + offset];
	// Store the result
	out[gindex] = result;
}
void fill_ints(int *x, int n)
{
	fill_n(x, n, 1);
}
int main(void)
{
	int *in, *out;	// host copies of a, b, c
	int *d_in, *d_out;	// device copies of a, b, c
	int size = (N + 2 *RADIUS) *sizeof(int);
	// Alloc space for host copies and setup values
	in = (int*) malloc(size);
	fill_ints(in, N + 2 *RADIUS);
	out = (int*) malloc(size);
	fill_ints(out, N + 2 *RADIUS);
	// Alloc space for device copies
	cudaMalloc((void **) &d_in, size);
	cudaMalloc((void **) &d_out, size);
	// Copy to device
	cudaMemcpy(d_in, in, size, cudaMemcpyHostToDevice);
	cudaMemcpy(d_out, out, size, cudaMemcpyHostToDevice);
	// Launch stencil_1d() kernel on GPU
	stencil_1d <<<(N + BLOCK_SIZE - 1 ) / BLOCK_SIZE, BLOCK_SIZE>>> (d_in + RADIUS, d_out + RADIUS);
	// Copy result back to host
	cudaMemcpy(out, d_out, size, cudaMemcpyDeviceToHost);
	// Cleanup
	free(in);
	free(out);
	cudaFree(d_in);
	cudaFree(d_out);
	return 0;
}

Overwriting o_stencil.cu


In [ ]:
!/usr/local/cuda/bin/nvcc -arch=sm_35 -rdc=true o_stencil.cu -o ./o_stencil -lcudadevrt
!nvprof ./o_stencil

nvcc warning : The 'compute_35', 'compute_37', 'compute_50', 'sm_35', 'sm_37' and 'sm_50' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).
==505== NVPROF is profiling process 505, command: ./o_stencil
==505== Warning: Auto boost enabled on device 0. Profiling results may be inconsistent.
==505== Profiling application: ./o_stencil
==505== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   63.68%  17.952us         2  8.9760us  8.8640us  9.0880us  [CUDA memcpy HtoD]
                   24.29%  6.8480us         1  6.8480us  6.8480us  6.8480us  [CUDA memcpy DtoH]
                   12.03%  3.3920us         1  3.3920us  3.3920us  3.3920us  stencil_1d(int*, int*)
      API calls:   99.46%  197.26ms         2  98.628ms  4.7020us  197.25ms  cudaMalloc
                    0.28%  559.56us         1  559.56us  559.56us  559.56us  cuDeviceTotalMem
           